In [1]:
from simtk import unit
from simtk import openmm
import numpy as np
from sys import stdout
from openmmtools import integrators
import random

Defines system for use in BG and in MD simulation, can create from scratch as commented out

In [2]:
# pdb = app.PDBFile('ala2_fromURL.pdb')
# topology = pdb.getTopology()
# positions = pdb.getPositions(asNumpy=True).value_in_unit(unit.nanometer)

# ff = app.ForceField('amber99sbildn.xml',"amber96_obc.xml")
# system = ff.createSystem(
#     topology=topology, 
#     removeCMMotion=True,
#     nonbondedMethod=app.NoCutoff,
#     constraints=app.HBonds, 
#     rigidWater=True
#     )

with open('ala2_xml_system.txt') as f:
    xml = f.read()
system = openmm.XmlSerializer.deserialize(xml)
#platform 2 = CUDA
platform = openmm.Platform.getPlatform(2)

In [3]:
#Setting up generator
import torch

device = "cuda:3" if torch.cuda.is_available() else "cpu"
dtype = torch.float32
# a context tensor to send data to the right device and dtype via '.to(ctx)'
ctx = torch.zeros([], device=device, dtype=dtype)

#need to load a dataset for dimensions of BG and to set up Mixed Coordinate Transform which requires data as an argument
import mdtraj
dataset = mdtraj.load('TSFtraj.dcd', top='ala2_fromURL.pdb')

import numpy as np
rigid_block = np.array([6, 8, 9, 10, 14])
z_matrix = np.array([
    [0, 1, 4, 6],
    [1, 4, 6, 8],
    [2, 1, 4, 0],
    [3, 1, 4, 0],
    [4, 6, 8, 14],
    [5, 4, 6, 8],
    [7, 6, 8, 4],
    [11, 10, 8, 6],
    [12, 10, 8, 11],
    [13, 10, 8, 11],
    [15, 14, 8, 16],
    [16, 14, 8, 6],
    [17, 16, 14, 15],
    [18, 16, 14, 8],
    [19, 18, 16, 14],
    [20, 18, 16, 19],
    [21, 18, 16, 19]
])

def dimensions(dataset):
        return np.prod(dataset.xyz[0].shape)
dim = dimensions(dataset)

#system setup, probably need to write a function to do this
# from simtk import openmm
# with open('ala2_xml_system.txt') as f:
#     xml = f.read()
# system = openmm.XmlSerializer.deserialize(xml)
from bgflow.distribution.energy.openmm import OpenMMBridge, OpenMMEnergy
temperature = 300.0 * unit.kelvin
collision_rate = 1.0 / unit.picosecond
timestep = 4.0 * unit.femtosecond
integrator = integrators.LangevinIntegrator(temperature=temperature,collision_rate=collision_rate,timestep=timestep)
energy_bridge = OpenMMBridge(system, integrator, n_workers=1)
target_energy = OpenMMEnergy(int(dim), energy_bridge)

#setting up training_data argument for MixedCoordinateTransform - not sure how much effect this has
n_train = len(dataset)//2
n_test = len(dataset) - n_train
permutation = np.random.permutation(n_train)
all_data = dataset.xyz.reshape(-1, dimensions(dataset))
training_data = torch.tensor(all_data[permutation]).to(ctx)
test_data = torch.tensor(all_data[permutation + n_train]).to(ctx)

import bgflow as bg

dim_cartesian = len(rigid_block) * 3 - 6
dim_bonds = len(z_matrix)
dim_angles = dim_bonds
dim_torsions = dim_bonds

#set up coordinate transform layer
coordinate_transform = bg.MixedCoordinateTransformation(
    data=training_data, 
    z_matrix=z_matrix,
    fixed_atoms=rigid_block,
    keepdims=dim_cartesian, 
    normalize_angles=True,
).to(ctx)

#setting up prior distribution
dim_ics = dim_bonds + dim_angles + dim_torsions + dim_cartesian
mean = torch.zeros(dim_ics).to(ctx) 
# passing the mean explicitly to create samples on the correct device
prior = bg.NormalDistribution(dim_ics, mean=mean)

split_into_ics_flow = bg.SplitFlow(dim_bonds, dim_angles, dim_torsions, dim_cartesian)

#defining RealNVP
class RealNVP(bg.SequentialFlow):
    
    def __init__(self, dim, hidden):
        self.dim = dim
        self.hidden = hidden
        super().__init__(self._create_layers())
    
    def _create_layers(self):
        dim_channel1 =  self.dim//2
        dim_channel2 = self.dim - dim_channel1
        split_into_2 = bg.SplitFlow(dim_channel1, dim_channel2)
        
        layers = [
            # -- split
            split_into_2,
            # --transform
            self._coupling_block(dim_channel1, dim_channel2),
            bg.SwapFlow(),
            self._coupling_block(dim_channel2, dim_channel1),
            # -- merge
            bg.InverseFlow(split_into_2)
        ]
        return layers
        
    def _dense_net(self, dim1, dim2):
        return bg.DenseNet(
            [dim1, *self.hidden, dim2],
            activation=torch.nn.ReLU()
        )
    
    def _coupling_block(self, dim1, dim2):
        return bg.CouplingFlow(bg.AffineTransformer(
            shift_transformation=self._dense_net(dim1, dim2),
            scale_transformation=self._dense_net(dim1, dim2)
        ))

#setting up normalising flow composed of RealNVP followed by coordinate transform
n_realnvp_blocks = 5
layers = []

for i in range(n_realnvp_blocks):
    layers.append(RealNVP(dim_ics, hidden=[128, 128, 128]))
layers.append(split_into_ics_flow)
layers.append(bg.InverseFlow(coordinate_transform))

flow = bg.SequentialFlow(layers).to(ctx)

#loading trained model into empty from
flow.load_state_dict(torch.load('modelTSFtraj_xmlsystem_20000KLL.pt'))

#setting up generator
generator = bg.BoltzmannGenerator(
    flow=flow,
    prior=prior,
    target=target_energy)

In [4]:
def getbg_positions():    
    bg_positions_tensor, dlogp_tensor = generator.sample(1,with_dlogp=True)
    bg_positions = bg_positions_tensor.cpu().detach().numpy().reshape(22,3)
    dlogp = dlogp_tensor.cpu().detach().numpy()
    return bg_positions
#print(bg_positions, dlogp)

In [6]:
#unit.BOLTZMANN_CONSTANT_kB is in units of J/K
kb = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
kt = kb * temperature/1000 /unit.joule*unit.kilojoule
beta = 1/kt

In [7]:
cycles = 100
MDsteps = 1000
BGmoves = 100

In [8]:
pdb = openmm.app.PDBFile('ala2_fromURL.pdb')
topology = pdb.getTopology()
positions = pdb.getPositions(asNumpy=True).value_in_unit(unit.nanometer)

md_temperature = 1000 * unit.kelvin
md_collision_rate = 1.0 / unit.picosecond
md_timestep = 1.0 * unit.femtosecond

integrator = integrators.LangevinIntegrator(temperature=md_temperature,collision_rate=md_collision_rate,timestep=md_timestep)
#integrator.setConstraintTolerance(0.00001)
#integrator = openmm.VerletIntegrator(timestep)
properties_dict = {}
properties_dict["DeviceIndex"] = "2"
simulation = openmm.app.Simulation(topology, system, integrator,platform,platformProperties=properties_dict)
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(10000)

In [9]:
#simulation.reporters.append(openmm.app.StateDataReporter(stdout, reportInterval=10, step=True, potentialEnergy=True,temperature=True,kineticEnergy=True))
for x in range(cycles):
    print('cycle',x)
    simulation.step(MDsteps)
    current_state = simulation.context.getState(getEnergy=True)
    current_total_energy = current_state.getKineticEnergy() + current_state.getPotentialEnergy()
    print('MD_end_energy',current_total_energy)
    for y in range(BGmoves):    
        integrator = integrators.LangevinIntegrator(temperature=md_temperature,collision_rate=md_collision_rate,timestep=md_timestep)
        bgsimulation = openmm.app.Simulation(topology,system,integrator,platform,platformProperties=properties_dict)
        bgsimulation.context.setPositions(getbg_positions())
        bgsimulation.context.setVelocitiesToTemperature(md_temperature)
        new_state = bgsimulation.context.getState(getEnergy=True)
        new_total_energy = new_state.getKineticEnergy() + new_state.getPotentialEnergy()
        #print('new_tot_energy',new_total_energy)
        energy_change = new_total_energy - current_total_energy
        acceptance_prob = min(1,np.exp(-beta*energy_change))
        if random.random() < acceptance_prob:
            print('accept new conformation')
            print('accepted BG energy',new_total_energy)
            new_checkpoint = bgsimulation.context.createCheckpoint()
            simulation.context.loadCheckpoint(new_checkpoint)
            print('loaded new checkpoint')
            break
        else:
            print('rejected BG energy',y,new_total_energy)


cycle 0
MD_end_energy 179.2122802734375 kJ/mol
rejected BG energy 0 400.13404846191406 kJ/mol
rejected BG energy 1 942.6077575683594 kJ/mol
rejected BG energy 2 429.0707702636719 kJ/mol
rejected BG energy 3 305.30584716796875 kJ/mol
rejected BG energy 4 435.0497741699219 kJ/mol
rejected BG energy 5 286.25 kJ/mol
accept new conformation
accepted BG energy 147.6786651611328 kJ/mol
loaded new checkpoint
cycle 1
MD_end_energy 255.29629516601562 kJ/mol
accept new conformation
accepted BG energy 202.2880859375 kJ/mol
loaded new checkpoint
cycle 2
MD_end_energy 271.8059997558594 kJ/mol
accept new conformation
accepted BG energy 258.56883239746094 kJ/mol
loaded new checkpoint
cycle 3
MD_end_energy 293.0 kJ/mol
accept new conformation
accepted BG energy 258.5285339355469 kJ/mol
loaded new checkpoint
cycle 4
MD_end_energy 365.2615966796875 kJ/mol
rejected BG energy 0 380.2794494628906 kJ/mol
rejected BG energy 1 476.8835144042969 kJ/mol
accept new conformation
accepted BG energy 204.1875 kJ/mol
